# RAG를 위한 5가지 청킹 전략
1. 고정 크기 분할 (Fixed-size chunking)
2. 의미 기반 분할 (Semantic Chunking)
3. 재귀적 분할 (Recursive chunking)
4. 문서 구조 기반 분할 (Document structure-based chunking)
5. LLM 기반 분할 (LLM-based chunking)

In [ ]:
%pip install openai nltk

## 1. 라이브러리 임포트 및 초기 설정

In [3]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
import re
import json
import numpy as np
import nltk

# OpenAI 클라이언트 초기화
client = OpenAI()

# NLTK 문장 토크나이저 다운로드 (최초 1회만 필요)
nltk.download('punkt')

# 헬퍼 함수: 결과 출력용
def print_chunks(strategy_name, chunks):
    """결과를 출력하는 함수"""
    print("="*20, f"전략 {strategy_name} 결과", "="*20)
    for i, chunk in enumerate(chunks):
        if isinstance(chunk, dict):
            print(f"[{i+1}번 청크]\n내용: {chunk['content']}\n메타데이터: {chunk['metadata']}\n" + "-"*50)
        else:
            print(f"[{i+1}번 청크]\n{chunk}\n" + "-"*50)
    print("\n")

[nltk_data] Downloading package punkt to /home/kubwa/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## 2. 샘플 데이터 정의

모든 예제에서 사용할 텍스트와 마크다운 데이터를 정의합니다.

In [5]:
# 일반 텍스트 샘플
sample_text = """
인공지능(AI)은 컴퓨터가 인간과 유사한 방식으로 생각하고, 학습하고, 문제를 해결할 수 있도록 하는 기술입니다. AI는 머신러닝, 딥러닝, 자연어 처리 등 다양한 하위 분야를 포함합니다.

머신러닝은 데이터로부터 패턴을 학습하여 예측이나 결정을 내리는 알고리즘을 연구하는 분야입니다. 예를 들어, 스팸 메일 필터는 머신러닝을 사용하여 원치 않는 이메일을 식별합니다.

딥러닝은 인공 신경망, 특히 심층 신경망을 사용하여 복잡한 패턴을 학습하는 머신러닝의 한 종류입니다. 이미지 인식, 음성 인식 등에서 뛰어난 성능을 보입니다. 딥러닝 모델인 GPT는 자연어 생성에 널리 사용됩니다.

자연어 처리(NLP)는 컴퓨터가 인간의 언어를 이해하고, 해석하고, 생성할 수 있도록 하는 AI의 한 분야입니다. 챗봇, 번역기, 감성 분석 등이 NLP의 대표적인 응용 사례입니다.
"""

# 마크다운 형식 샘플
sample_markdown = """
# RAG 파이프라인의 이해

## 1. 검색 (Retrieval)
검색 단계는 사용자의 질문과 가장 관련성이 높은 문서를 벡터 데이터베이스에서 찾는 과정입니다. 임베딩 모델을 사용하여 질문과 문서를 벡터로 변환하고, 코사인 유사도를 통해 관련성을 계산합니다.

## 2. 증강 (Augmentation)
증강 단계에서는 검색된 문서를 프롬프트에 추가하여 LLM에 전달할 컨텍스트를 보강합니다. 이 과정을 통해 LLM은 더 정확하고 풍부한 답변을 생성할 수 있습니다.

## 3. 생성 (Generation)
생성 단계에서 LLM은 증강된 프롬프트를 바탕으로 최종 답변을 생성합니다. 검색된 외부 지식을 활용하므로, 환각(Hallucination) 현상을 크게 줄일 수 있습니다.
"""

## 전략 1: 고정 크기 분할 (Fixed-size chunking)

단순히 텍스트를 정해진 크기로 자르는 방식입니다.

In [6]:
def fixed_size_chunking(text, chunk_size, chunk_overlap):
    """고정 크기 청킹 구현"""
    chunks = []
    start_index = 0
    while start_index < len(text):
        end_index = start_index + chunk_size
        chunks.append(text[start_index:end_index])
        start_index += chunk_size - chunk_overlap
    return chunks

chunks_1 = fixed_size_chunking(sample_text, chunk_size=150, chunk_overlap=30)
print_chunks("1: 고정 크기 분할", chunks_1)

==================== 전략 1: 고정 크기 분할 결과 ====================
[1번 청크]

인공지능(AI)은 컴퓨터가 인간과 유사한 방식으로 생각하고, 학습하고, 문제를 해결할 수 있도록 하는 기술입니다. AI는 머신러닝, 딥러닝, 자연어 처리 등 다양한 하위 분야를 포함합니다.

머신러닝은 데이터로부터 패턴을 학습하여 예측이나 결정을 내리는 알고리즘을 연
--------------------------------------------------
[2번 청크]
 패턴을 학습하여 예측이나 결정을 내리는 알고리즘을 연구하는 분야입니다. 예를 들어, 스팸 메일 필터는 머신러닝을 사용하여 원치 않는 이메일을 식별합니다.

딥러닝은 인공 신경망, 특히 심층 신경망을 사용하여 복잡한 패턴을 학습하는 머신러닝의 한 종류입니다. 이미지 인
--------------------------------------------------
[3번 청크]
 패턴을 학습하는 머신러닝의 한 종류입니다. 이미지 인식, 음성 인식 등에서 뛰어난 성능을 보입니다. 딥러닝 모델인 GPT는 자연어 생성에 널리 사용됩니다.

자연어 처리(NLP)는 컴퓨터가 인간의 언어를 이해하고, 해석하고, 생성할 수 있도록 하는 AI의 한 분야입니
--------------------------------------------------
[4번 청크]
 해석하고, 생성할 수 있도록 하는 AI의 한 분야입니다. 챗봇, 번역기, 감성 분석 등이 NLP의 대표적인 응용 사례입니다.

--------------------------------------------------




## 전략 2: 의미 기반 분할 (Semantic Chunking)

문장 임베딩 간의 코사인 유사도를 기반으로 의미적으로 연결된 문장들을 하나의 청크로 묶습니다.

In [7]:
def get_embeddings(texts, model="text-embedding-3-small"):
    """OpenAI API를 사용하여 텍스트 목록의 임베딩을 가져오는 함수"""
    response = client.embeddings.create(input=texts, model=model)
    return [np.array(item.embedding) for item in response.data]

def custom_cosine_similarity(vec1, vec2):
    """두 벡터 간의 코사인 유사도를 계산합니다."""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

def semantic_chunking(text, similarity_threshold=0.3):
    """의미 기반 청킹 구현"""
    sentences = nltk.sent_tokenize(text)
    if not sentences:
        return []

    embeddings = get_embeddings(sentences)
    chunks = []
    current_chunk_sentences = [sentences[0]]

    for i in range(len(sentences) - 1):
        similarity = custom_cosine_similarity(embeddings[i], embeddings[i+1])
        if similarity >= similarity_threshold:
            current_chunk_sentences.append(sentences[i+1])
        else:
            chunks.append(" ".join(current_chunk_sentences))
            current_chunk_sentences = [sentences[i+1]]

    chunks.append(" ".join(current_chunk_sentences))
    return chunks

chunks_2 = semantic_chunking(sample_text)
print_chunks("2: 의미 기반 분할", chunks_2)

==================== 전략 2: 의미 기반 분할 결과 ====================
[1번 청크]

인공지능(AI)은 컴퓨터가 인간과 유사한 방식으로 생각하고, 학습하고, 문제를 해결할 수 있도록 하는 기술입니다. AI는 머신러닝, 딥러닝, 자연어 처리 등 다양한 하위 분야를 포함합니다. 머신러닝은 데이터로부터 패턴을 학습하여 예측이나 결정을 내리는 알고리즘을 연구하는 분야입니다. 예를 들어, 스팸 메일 필터는 머신러닝을 사용하여 원치 않는 이메일을 식별합니다. 딥러닝은 인공 신경망, 특히 심층 신경망을 사용하여 복잡한 패턴을 학습하는 머신러닝의 한 종류입니다.
--------------------------------------------------
[2번 청크]
이미지 인식, 음성 인식 등에서 뛰어난 성능을 보입니다.
--------------------------------------------------
[3번 청크]
딥러닝 모델인 GPT는 자연어 생성에 널리 사용됩니다. 자연어 처리(NLP)는 컴퓨터가 인간의 언어를 이해하고, 해석하고, 생성할 수 있도록 하는 AI의 한 분야입니다. 챗봇, 번역기, 감성 분석 등이 NLP의 대표적인 응용 사례입니다.
--------------------------------------------------




## 전략 3: 재귀적 분할 (Recursive chunking)

`\n\n`, `\n`, `     ` 등 구분자 목록을 순서대로 적용해 계층적으로 텍스트를 나눕니다.

In [8]:
def recursive_chunking(text, separators, chunk_size):
    """순수 Python으로 재귀적 청킹 구현"""
    final_chunks = []
    primary_chunks = text.split(separators[0])

    for chunk in primary_chunks:
        if len(chunk) > chunk_size and len(separators) > 1:
            sub_chunks = recursive_chunking(chunk, separators[1:], chunk_size)
            final_chunks.extend(sub_chunks)
        else:
            if chunk.strip():
                final_chunks.append(chunk)
    return final_chunks

chunks_3 = recursive_chunking(sample_text, separators=["\n\n", "\n", " "], chunk_size=200)
print_chunks("3: 재귀적 분할", chunks_3)

==================== 전략 3: 재귀적 분할 결과 ====================
[1번 청크]

인공지능(AI)은 컴퓨터가 인간과 유사한 방식으로 생각하고, 학습하고, 문제를 해결할 수 있도록 하는 기술입니다. AI는 머신러닝, 딥러닝, 자연어 처리 등 다양한 하위 분야를 포함합니다.
--------------------------------------------------
[2번 청크]
머신러닝은 데이터로부터 패턴을 학습하여 예측이나 결정을 내리는 알고리즘을 연구하는 분야입니다. 예를 들어, 스팸 메일 필터는 머신러닝을 사용하여 원치 않는 이메일을 식별합니다.
--------------------------------------------------
[3번 청크]
딥러닝은 인공 신경망, 특히 심층 신경망을 사용하여 복잡한 패턴을 학습하는 머신러닝의 한 종류입니다. 이미지 인식, 음성 인식 등에서 뛰어난 성능을 보입니다. 딥러닝 모델인 GPT는 자연어 생성에 널리 사용됩니다.
--------------------------------------------------
[4번 청크]
자연어 처리(NLP)는 컴퓨터가 인간의 언어를 이해하고, 해석하고, 생성할 수 있도록 하는 AI의 한 분야입니다. 챗봇, 번역기, 감성 분석 등이 NLP의 대표적인 응용 사례입니다.

--------------------------------------------------




## 전략 4: 문서 구조 기반 분할 (Document structure-based chunking)

마크다운의 제목 (`#`, `##`)과 같은 문서의 구조를 기준으로 청크를 생성합니다.

In [9]:
def markdown_chunking(markdown_text):
    """정규표현식을 사용하여 마크다운 구조 기반 청킹 구현"""
    pattern = r'(^#+\s.*$)'
    sections = re.split(pattern, markdown_text, flags=re.MULTILINE)

    chunks = []
    for i in range(1, len(sections), 2):
        header = sections[i].strip()
        content = sections[i+1].strip()
        level = header.find(' ')
        header_type = f"Header {level}"
        chunks.append({
            "content": f"{header}\n{content}",
            "metadata": {header_type: header.lstrip('# ').strip()}
        })
    return chunks

chunks_4 = markdown_chunking(sample_markdown)
print_chunks("4: 문서 구조 기반 분할", chunks_4)

==================== 전략 4: 문서 구조 기반 분할 결과 ====================
[1번 청크]
내용: # RAG 파이프라인의 이해

메타데이터: {'Header 1': 'RAG 파이프라인의 이해'}
--------------------------------------------------
[2번 청크]
내용: ## 1. 검색 (Retrieval)
검색 단계는 사용자의 질문과 가장 관련성이 높은 문서를 벡터 데이터베이스에서 찾는 과정입니다. 임베딩 모델을 사용하여 질문과 문서를 벡터로 변환하고, 코사인 유사도를 통해 관련성을 계산합니다.
메타데이터: {'Header 2': '1. 검색 (Retrieval)'}
--------------------------------------------------
[3번 청크]
내용: ## 2. 증강 (Augmentation)
증강 단계에서는 검색된 문서를 프롬프트에 추가하여 LLM에 전달할 컨텍스트를 보강합니다. 이 과정을 통해 LLM은 더 정확하고 풍부한 답변을 생성할 수 있습니다.
메타데이터: {'Header 2': '2. 증강 (Augmentation)'}
--------------------------------------------------
[4번 청크]
내용: ## 3. 생성 (Generation)
생성 단계에서 LLM은 증강된 프롬프트를 바탕으로 최종 답변을 생성합니다. 검색된 외부 지식을 활용하므로, 환각(Hallucination) 현상을 크게 줄일 수 있습니다.
메타데이터: {'Header 2': '3. 생성 (Generation)'}
--------------------------------------------------




## 전략 5: LLM 기반 분할 (LLM-based chunking)

LLM에게 직접 프롬프트를 보내 텍스트를 의미 단위로 분할하도록 요청합니다.

In [10]:
def llm_based_chunking(text_to_chunk):
    """OpenAI API를 직접 사용하여 LLM 기반 청킹 구현"""
    system_prompt = "You are an expert in text chunking. Your task is to split the text into semantically meaningful and self-contained chunks. Return the result as a JSON object with a single key 'chunks' containing a list of strings."

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text_to_chunk}
        ],
        response_format={"type": "json_object"},
        temperature=0
    )
    result_json = response.choices[0].message.content
    result_dict = json.loads(result_json)
    return result_dict.get('chunks', [])

chunks_5 = llm_based_chunking(sample_text)
print_chunks("5: LLM 기반 분할", chunks_5)

==================== 전략 5: LLM 기반 분할 결과 ====================
[1번 청크]
인공지능(AI)은 컴퓨터가 인간과 유사한 방식으로 생각하고, 학습하고, 문제를 해결할 수 있도록 하는 기술입니다. AI는 머신러닝, 딥러닝, 자연어 처리 등 다양한 하위 분야를 포함합니다.
--------------------------------------------------
[2번 청크]
머신러닝은 데이터로부터 패턴을 학습하여 예측이나 결정을 내리는 알고리즘을 연구하는 분야입니다. 예를 들어, 스팸 메일 필터는 머신러닝을 사용하여 원치 않는 이메일을 식별합니다.
--------------------------------------------------
[3번 청크]
딥러닝은 인공 신경망, 특히 심층 신경망을 사용하여 복잡한 패턴을 학습하는 머신러닝의 한 종류입니다. 이미지 인식, 음성 인식 등에서 뛰어난 성능을 보입니다. 딥러닝 모델인 GPT는 자연어 생성에 널리 사용됩니다.
--------------------------------------------------
[4번 청크]
자연어 처리(NLP)는 컴퓨터가 인간의 언어를 이해하고, 해석하고, 생성할 수 있도록 하는 AI의 한 분야입니다. 챗봇, 번역기, 감성 분석 등이 NLP의 대표적인 응용 사례입니다.
--------------------------------------------------


